In [17]:
import pandas as pd
import sqlite3

# Load CSV files
prods = pd.read_csv("PRODUCTS_TAKEHOME.csv")
trans = pd.read_csv("TRANSACTION_TAKEHOME.csv")
users = pd.read_csv("USER_TAKEHOME.csv")



# Set up in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load DataFrames into SQLite tables
prods.to_sql("products", conn, index=False, if_exists="replace")
trans.to_sql("transactions", conn, index=False, if_exists="replace")
users.to_sql("users", conn, index=False, if_exists="replace")

# Cleaning PRODUCTS data, remove duplicates, remove nulls, change barcode data format from scientific notations to string

conn.execute( """CREATE TABLE products_cleaned AS 
SELECT CAST( barcode AS TEXT) AS cleanBarcode, CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, BRAND
FROM products
WHERE BARCODE IS NOT NULL
GROUP BY CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, BRAND""") 

# Cleaning TRANSACTIONS data, change barcode data format from scientific notations to string, removes erroneous zero lines,
conn.execute("""CREATE TABLE transactions_cleaned AS 
SELECT 
    CAST( barcode AS TEXT) AS cleanBarcode, 
    RECEIPT_ID, 
    DATETIME(PURCHASE_DATE) AS PURCHASE_DATE, 
    DATETIME(SCAN_DATE) AS SCAN_DATE, STORE_NAME, TRIM(CAST( USER_ID AS TEXT)) AS USER_ID,
    FINAL_QUANTITY AS cleanQty, FINAL_SALE
    
FROM transactions
WHERE FINAL_QUANTITY != 'zero' AND 
        (CAST( barcode AS TEXT) IS NOT NULL AND CAST( barcode AS TEXT) NOT IN (-1.0,'-1.0'))
GROUP BY CAST( barcode AS TEXT), RECEIPT_ID, DATETIME(PURCHASE_DATE), DATETIME(SCAN_DATE), 
             STORE_NAME, TRIM(CAST( USER_ID AS TEXT)), FINAL_QUANTITY, FINAL_SALE""") 


# Cleaning USERS data, remove duplicates, remove nulls

conn.execute( """CREATE TABLE users_cleaned AS 
SELECT 
    TRIM(CAST(ID AS TEXT)) AS ID, 
    DATETIME(CREATED_DATE) AS CREATED_DATE, 
    DATETIME(BIRTH_DATE) AS BIRTH_DATE,
    STATE, LANGUAGE, GENDER
FROM users
WHERE ID IS NOT NULL
GROUP BY TRIM(ID), DATETIME(CREATED_DATE), DATETIME(BIRTH_DATE), STATE, LANGUAGE, GENDER""") 

conn.commit()

## Here are a my queries. This is where I am testing the results of my new tables

###Testing the product file
queryA = """ SELECT CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4
FROM products_cleaned
WHERE CATEGORY_2 like '%Dips & Salsa%'
GROUP BY CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4
ORDER BY CATEGORY_2
LIMIT 100
"""

###Testing the users file
queryB = """ SELECT *
FROM users_cleaned
LIMIT 100
"""

###Testing the transactions file
query1 = """ SELECT COUNT(*), MIN(PURCHASE_DATE), MAX(PURCHASE_DATE)
FROM transactions_cleaned
WHERE cleanBarcode is null
LIMIT 10
"""

###This answers: Which is the leading brand in the Dips & Salsa category?
queryD = """SELECT BRAND, SUM(cleanQty), SUM(FINAL_SALE), COUNT(*)
FROM transactions_cleaned trs left join products_cleaned prd on trs.cleanBarcode = prd.cleanBarcode
WHERE CATEGORY_2 like '%Dips & Salsa%'
GROUP BY BRAND
ORDER BY SUM(FINAL_SALE) DESC
LIMIT 1"""

###This answers: Who are Fetch’s power users?

queryE = """ SELECT USER_ID, SUM(cleanQty), SUM(FINAL_SALE)
FROM transactions_cleaned
GROUP BY USER_ID
ORDER BY SUM(FINAL_SALE) DESC
LIMIT 5
"""

###This answers: What are the top 5 brands by receipts scanned among users 21 and over?
queryF = """SELECT BRAND, COUNT(DISTINCT RECEIPT_ID) AS UniqueReceipts
FROM transactions_cleaned trs 
INNER JOIN products_cleaned prd on trs.cleanBarcode = prd.cleanBarcode
INNER JOIN users_cleaned urs on ID = USER_ID
WHERE CAST((julianday('now') - julianday(date(BIRTH_DATE))) / 365.25 AS INTEGER) >= 21
GROUP BY BRAND
ORDER BY COUNT(DISTINCT RECEIPT_ID) DESC
LIMIT 5
"""

Top5Brands = pd.read_sql_query(queryF, conn)
PowerUsers = pd.read_sql_query(queryE, conn)
DipsNSalsa = pd.read_sql_query(queryD, conn)
print(Top5Brands)
print(PowerUsers)
print(DipsNSalsa)


              BRAND  UniqueReceipts
0  DEAN'S DAIRY DIP               1
                    USER_ID  SUM(cleanQty)  SUM(FINAL_SALE)
0  65e4bc2716cc391732143569            9.0            88.37
1  60a5363facc00d347abadc8e           44.0            84.58
2  6183300cf998e47aad2d6f5d           11.1            82.28
3  6475fd16a55bb77a0e279ee0            4.0            77.80
4  643059f0838dd2651fb27f50            4.0            75.99
              BRAND  SUM(cleanQty)  SUM(FINAL_SALE)  COUNT(*)
0  DEAN'S DAIRY DIP           22.0            39.95        22
